In [1]:
from html.parser import HTMLParser
from pprint import pprint

import pandas as pd
import copy

import urllib.request
import urllib.error
from urllib.parse import urljoin, urlparse
from urllib.robotparser import RobotFileParser
from urllib.request import urlopen
from urllib.error import URLError, HTTPError


class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attr):
        global start_tag
        global temp_links
        global domain
        start_tag = tag
        if tag == 'a':
            for item in attr:
                # Get links
                if item[0] == 'href':
                    # href="/abc"
                    if item[1].startswith("/"):
                        temp_links += ['https://' + domain + item[1]]
                    # href="domain/abc"
                    elif item[1].startswith(domain):
                        temp_links += [item[1]]
        temp_links = list(set(temp_links))

    def handle_data(self, data):
        global start_tag
        global all_data
        if start_tag != 'script' and start_tag != 'style':
            data = data.rstrip().lstrip()
            if len(data) > 0:
                all_data += [data]


# Dictionary to save parsed robots.txt files
ROBOTS = dict()


def robot_check(url):
    netloc = urlparse(url).netloc
    if netloc not in ROBOTS:
        robotsurl = urljoin(url, '/robots.txt')
        ROBOTS[netloc] = RobotFileParser(robotsurl)
        try:
            ROBOTS[netloc].read()
        except:
            print(netloc)
    return ROBOTS[netloc].can_fetch('*', url)


def get_page(url):
    if robot_check(url):
        try:
            fd = urlopen(url)
            content = fd.read()
            fd.close()
            return content.decode('utf8')
        except:
            return ''
    else:
        print("Can't fetch data from " + url)
        return ''


def deep(d=2):
    global parser
    global all_links
    global temp_links
    global url_limit
    global domain
    for i in range(d):
        all_links = copy.deepcopy(temp_links)
        temp_links = []
        print("Deep:", i + 1)
        print(len(all_links))
        if url_limit > 0:
            for l in all_links:
                url_limit -= 1
                # print(url_limit)
                if url_limit <= 0:
                    break
                str_raw = get_page(l)
                if isinstance(str_raw, str) or str == '':
                    parser.feed(str_raw)
        print("End deep:", i + 1)


# URL = 'https://www.bbc.co.uk/iplayer'
start_tag = ''
all_links = []
temp_links = []
all_data = []
url_limit = 50
parser = MyHTMLParser()
domain = ''
input_data = []


def run(URL):
    global parser
    global domain
    global input_data
    global all_data
    domain = urlparse(URL).netloc
    parser.feed(get_page(URL))
    deep()
    # for d in all_data:
    #   input_data += d.split(' ')
    input_data = ' '.join(list(all_data))
    return input_data


In [ ]:
usecols = ['url', 'category']
data = pd.read_csv('urls.csv', usecols=usecols)
data.head()

In [ ]:
data['text'] = ''
data.to_csv('dataset.csv', index=False)
for i, v in data.loc[:, 'url'].items():
    print("########### Fetch link", i, ":", v, "###########")
    try:
        text = run(v)
        data.at[i, 'text'] = text
    except:
        data.at[i, 'text'] = 'ERROR'
    data.to_csv('dataset.csv', index=False)


In [2]:
print(run('https://ratiocoffee.com/'))

ratiocoffee.com
Can't fetch data from https://ratiocoffee.com/
Deep: 1
0
End deep: 1
Deep: 2
0
End deep: 2

